**Note**
Order kurang dari 40 jangan didelete. bisa jadi kurang aja angkanya

#Upload Data

In [15]:
import pandas as pd
import numpy as np
import os
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [16]:
# Suppress SettingWithCopyWarning, nutupin warning
pd.options.mode.chained_assignment = None  # default='warn'

#max row view
pd.set_option('display.max_rows', None)

In [17]:
#df = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check1.csv", delimiter=';')

# #Jika dibagi 2 excel (Jogja dan Semarang)
df1 = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check1.csv", delimiter=';')
df2 = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check2.csv", delimiter=';')
# # df3 = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check3.csv", delimiter=';')

# Combine dataframes without keys, tambahkan df3 jika ada 3
df = pd.concat([df1, df2], ignore_index=True)


# Reset the index to remove the extra level
df = df.reset_index(drop=True)

<ipython-input-17-3f4cd774ec27>:4: DtypeWarning: Columns (1,5,10,12,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check1.csv", delimiter=';')
<ipython-input-17-3f4cd774ec27>:5: DtypeWarning: Columns (1,10,12,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check2.csv", delimiter=';')


In [18]:
# Print the total number of rows
total_rows = df.shape[0]
print(f'Total rows in the DataFrame: {total_rows}')

Total rows in the DataFrame: 1106380


In [19]:
# Add a new empty column named "New Column"
df['Keterangan'] = None

#Processing Data 1

In [ ]:
df.head()

##Check Order Id & No Telpon

In [21]:
def process_order_id(df):
    # Delete rows where 'ORDER ID' has less than 40 characters
    df = df[df['ORDER ID'].str.len() >= 35]

    # Check if 'ORDER ID' length is less than 40
    df.loc[df['ORDER ID'].str.len() < 40, 'Keterangan'] = 'order id salah'

    # Update 'Keterangan' for 'ORDER ID' with more than 40 characters
    df.loc[df['ORDER ID'].str.len() > 40, 'Keterangan'] = df.apply(
        lambda row: row['Keterangan'] + ' & order id salah' if pd.notna(row['Keterangan']) else 'order id salah',
        axis=1
    )

    # Check 'NO. HP' and update 'Keterangan' if the condition is not met
    df.loc[~df['NO. HP'].astype(str).str.startswith(('6', '8', '0')), 'Keterangan'] = df.apply(
        lambda row: row['Keterangan'] + ' & Telpon Check' if pd.notna(row['Keterangan']) else 'no telpon salah',
        axis=1
    )

    return df

In [22]:
# Call the function with the DataFrame
df = process_order_id(df)

In [ ]:
# View rows where 'Keterangan' is not null
result_df = df[df['Keterangan'].notnull()]

# Display the result
result_df

In [24]:
ori_df = df

##Formatting Date & Delete Source Data

In [25]:
df.head()

,ORDER ID,NO. HP,Doc Type,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,...,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Source Data,Remarks Zolos Incomplete,Unnamed: 19,Keterangan
0,20240807111215800000DANAW3ID705792020428,6281367226124,KTP,CHECK,NaN,NaN,agent.jogja407@radikari.com,Wibulan Ekaning Ratri,08-08-2024 13:51:02,KK,...,0.0,08-08-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,20240808111215800000DANAW3ID518791941028,6282120783691,KTP,REVOKED,FOTO KTP TEMPELAN,NaN,agent.jogja407@radikari.com,Wibulan Ekaning Ratri,08-08-2024 13:59:08,Tidak Ada,...,0.0,08-08-2024,No,REVOKED,YOUR ID CARD IS NOT EKTP/KTP,Photo On Id Card Taped (Foto KTP Tempelan),NaN,NaN,NaN,None
2,20240808111215800000DANAW3ID592891813060,6285362928646,KTP,REJECT,BACKGROUND TIDAK SESUAI TTL,NaN,agent.jogja407@radikari.com,Wibulan Ekaning Ratri,08-08-2024 14:06:50,KK,...,0.0,08-08-2024,No,REVOKED,Reject document doesn't match the specified ty...,Background KTP doesn’t match (Background Foto ...,NaN,NaN,NaN,None
3,20240807111215800000DANAW3ID490093092346,6281370744374,KTP,CHECK,NaN,NaN,agent.jogja403@radikari.com,Yunita Risang Saksana Awiati,08-08-2024 13:47:29,KK,...,0.0,08-08-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,20240808111215800000DANAW3ID247793732769,6282290386265,KTP,REJECT,BACKGROUND TIDAK SESUAI TTL,NaN,agent.jogja403@radikari.com,Yunita Risang Saksana Awiati,08-08-2024 13:55:03,KK,...,0.0,08-08-2024,No,REVOKED,Reject document doesn't match the specified ty...,Background KTP doesn’t match (Background Foto ...,NaN,NaN,NaN,None


In [26]:
# df['NO. HP'] = df['NO. HP'].astype('int64')
df['ORDER ID'] = df['ORDER ID'].astype('str')

#date
# df['Modified Date'] = pd.to_datetime(df['Modified Date'], errors='coerce')
df['Modified Date'] = pd.to_datetime(df['Modified Date'], format='%d-%m-%Y %H:%M:%S', errors='coerce')

df['WorkingDate'] = pd.to_datetime(df['WorkingDate'], format='%d-%m-%Y', errors='coerce')
#df['WorkingDate'] = df['WorkingDate'].dt.strftime('%Y-%m-%d')

# the column containing extra spaces
df['ORDER ID'] = df['ORDER ID'].str.strip()

# df['NO. HP'] = df['NO. HP'].astype('int64')
df['Face Score'] = df['Face Score'].astype('str')

#Doc Type
df['Doc Type'] = df['Doc Type'].astype('str')

df.drop('Source Data', axis=1, inplace=True)


###Melakukan filter Start Date & End Date pada Working Date

In [27]:
oldest_date = df['WorkingDate'].min()
latest_date = df['WorkingDate'].max()
print("The oldest date is:", oldest_date)
print("The oldest date is:", latest_date)

The oldest date is: 2024-08-01 00:00:00
The oldest date is: 2024-08-25 00:00:00


In [28]:
from google.colab import files

try:
  # Take user input for start and end dates
  print("The oldest date is:", oldest_date)
  print("The oldest date is:", latest_date)
  start_date = input("Enter start date (YYYY-MM-DD): ")
  end_date = input("Enter end date (YYYY-MM-DD): ")

  # Convert input dates to datetime objects
  start_date = pd.to_datetime(start_date)
  end_date = pd.to_datetime(end_date)

  # Filter DataFrame based on the input dates
  df = df[(df['WorkingDate'] >= start_date) & (df['WorkingDate'] <= end_date)]

except ValueError as e:
    # Handle the ValueError (e.g., invalid date format)
    print(f"Error: {e}. Please enter dates in the 'YYYY-MM-DD' format.")
except Exception as e:
    # Handle other exceptions
    print(f"An unexpected error occurred: {e}")

The oldest date is: 2024-08-01 00:00:00
The oldest date is: 2024-08-25 00:00:00
Enter start date (YYYY-MM-DD): 2024-8-1
Enter end date (YYYY-MM-DD): 2024-8-24


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1080615 entries, 0 to 1105580
Data columns (total 21 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   ORDER ID                  1080615 non-null  object        
 1   NO. HP                    1080614 non-null  object        
 2   Doc Type                  1080615 non-null  object        
 3   STATUS                    1080588 non-null  object        
 4   Reason                    167966 non-null   object        
 5   New Remarks               1 non-null        object        
 6   ID AGENT                  1080614 non-null  object        
 7   NAMA AGENT                1080615 non-null  object        
 8   Modified Date             1079702 non-null  datetime64[ns]
 9   Support DOC               1079738 non-null  object        
 10  Face Score                1080615 non-null  object        
 11  OCR                       1079737 non-null  object     

In [ ]:
df.head()

## Check Status

In [31]:
def update_status(row):

    if pd.isna(row['STATUS']) and row['WorkingDate'] == end_date:
      return row['Keterangan']

    elif pd.isna(row['STATUS']) or row['STATUS'] not in ['CHECK', 'REJECT', 'REVOKED','REJECT BY SYSTEM']:
      if pd.isna(row['Keterangan']):
        return 'check status'
      else:
        return f"{row['Keterangan']} & check status"

    elif row['STATUS'] == 'CHECK' and pd.notna(row['Reason']):
      if pd.isna(row['Keterangan']):
        return 'check status'
      else:
        return f"{row['Keterangan']} & check status"

    elif row['STATUS'] == 'CHECK' and pd.notna(row['Status Revoke']):
      if pd.isna(row['Keterangan']):
        return 'check status'
      else:
        return f"{row['Keterangan']} & check status"

    elif row['STATUS'] in ['REJECT', 'REVOKED'] and pd.isna(row['Reason']):
      if pd.isna(row['Keterangan']):
        return 'check status'
      else:
        return f"{row['Keterangan']} & check status"

    # elif row['STATUS'] in ['REJECT BY SYSTEM'] and pd.notna(row['Reason']) and and pd.notna(row['Reason']):
    #   if pd.isna(row['Keterangan']):
    #     return 'check status'
    #   else:
    #     return f"{row['Keterangan']} & check status"

    else:
        return row['Keterangan']

In [32]:
# Apply the custom function to update "Keterangan" column
df['Keterangan'] = df.apply(update_status, axis=1)

In [33]:
# Remove values in 'Keterangan' column where 'STATUS' is "REJECT BY SYSTEM"
df.loc[df['STATUS'] == 'REJECT BY SYSTEM', 'Keterangan'] = np.nan

##NEW Check ID Agent

In [34]:
# rumus excel:  =TEXTJOIN(", "; TRUE; """" & A1:A250 & """ : """ & B1:B250 & """")
# Dictionary of agent IDs and names
agent_dict = {

"agent.jogja403@radikari.com" : "Yunita Risang Saksana Awiati", "agent.jogja407@radikari.com" : "Wibulan Ekaning Ratri", "kwijaya@radikari.com" : "Krisna Wijaya", "agent.jogja435@radikari.com" : "Ryan Oktavianto", "agent.jogja438@radikari.com" : "Chatarina Ria Septiana", "wiwiks@radikari.com" : "Wiwik Suharti", "agent.jogja487@radikari.com" : "Teguh Bagus Joko Wahono", "agent.jogja511@radikari.com" : "Hesti Vera", "agent.jogja517@radikari.com" : "Arlinda Elsa Ulvia", "agent.jogja566@radikari.com" : "Eri Hardani", "amalia.nurhayati@radikari.com" : "Amalia Nurhayati", "agent.jogja615@radikari.com" : "Puji Kasih Santika", "agent.jogja674@radikari.com" : "Wiwid prastianto", "agent.jogja738@radikari.com" : "Yudha Tantra Hanurizal", "agent.jogja740@radikari.com" : "Bobby Yulianto", "agent.jogja745@radikari.com" : "Rio Jati Kumbara", "agent.jogja749@radikari.com" : "Vincentius Juan Fergi Hermantya", "agent.jogja751@radikari.com" : "Ahda Rasyidi Ahmad", "agent.jogja769@radikari.com" : "Galih Reka Mahendra", "agent.jogja770@radikari.com" : "Galih Fajar Kurniawan", "agent.jogja784@radikari.com" : "Rika Ferawati", "agent.jogja814@radikari.com" : "Regitha Aprilia Ochie", "agent.jogja876@radikari.com" : "Yuriatun", "amandarapih@radikari.com" : "Amanda Rapih Subarja Putri", "nabilakhansa@radikari.com" : "Nabila Khansa Afanin", "paiznurdiansyah@radikari.com" : "Paiz Nurdiansyah", "yuliastutik@radikari.com" : "Yuli Astutik", "agent.jogja897@radikari.com" : "Afwan Hilmi Muttaqin", "agent.jogja899@radikari.com" : "Elysama Tamba", "agent.jogja902@radikari.com" : "Mahmudi Waruwu", "agent.jogja905@radikari.com" : "Ridhol Fatla", "agent.jogja907@radikari.com" : "Septi Sindisat Rahayu", "agent.jogja908@radikari.com" : "Siti Ulfah Hardiyanti", "agent.jogja909@radikari.com" : "Slamet Setiyono", "agent.jogja910@radikari.com" : "Topan Jiwodewo", "agent.jogja912@radikari.com" : "Dea Fatika Putri", "agent.jogja917@radikari.com" : "Raden Kurnianto Ridwan Hanafi", "agent.jogja918@radikari.com" : "Randy Aldany Barasa", "agent.jogja920@radikari.com" : "Satria Bobby Fernando", "agent.jogja922@radikari.com" : "Vivia Tri Ayuningtyas", "agent.jogja924@radikari.com" : "Zarah Usra", "agent.jogja925@radikari.com" : "Abiyan Roofif Mahendra", "agent.jogja928@radikari.com" : "Donie Pramudita", "agent.jogja929@radikari.com" : "Eka Edi Pratama", "agent.jogja930@radikari.com" : "Firman Syahputra", "agent.jogja931@radikari.com" : "Hanifah Argiyanti Kusumaningrum", "agent.jogja932@radikari.com" : "Irvan Susilo Widjiatmoko", "agent.jogja937@radikari.com" : "Andreas Suryo Prasetyo", "agent.jogja940@radikari.com" : "Krisna Dewa Nanda", "agent.jogja942@radikari.com" : "Mella Sutami", "agent.jogja945@radikari.com" : "Rido Mesia Sumunar", "agent.jogja946@radikari.com" : "Wahyu Adi Purnomo", "agent.jogja951@radikari.com" : "Lailatul Rahmah", "agent.jogja952@radikari.com" : "Lintang Wahyu Herlambang", "agent.jogja953@radikari.com" : "Manggala Maulana Mahardhika", "agent.jogja954@radikari.com" : "Ngrayong Agung Nugroho", "agent.jogja955@radikari.com" : "Nova Putra Alves Correia", "agent.jogja957@radikari.com" : "Pratiwi Matoka", "alfredo.yunara@radikari.com" : "Alfredo Yunora", "arbinugraha@radikari.com" : "Arbi Tama Nugraha", "ariffabikhan@radikari.com" : "Arif Fabikhan", "bagusramadhan@radikari.com" : "Bagus Ramadhan Firdausi", "bayuasmoro@radikari.com" : "Bayu Asmoro Putro", "bimasatriyapahlawan@radikari.com" : "Bima Satriya Pahlawan", "cintapratiwi@radikari.com" : "Cinta Pratiwi Taasiringan", "dandiaditya@radikari.com" : "Dandi Aditya Rahmanto", "davidfauzan@radikari.com" : "David Fauzan", "dewipujilestari@radikari.com" : "Dewi Puji Lestari", "deyskeerika@radikari.com" : "Deyske Erika Rembaen", "dhenisafdhaly@radikari.com" : "Dheni Nandar Safdhaly", "ekonurjayadi@radikari.com" : "Eko Nur Jayadi", "faliaputri@radikari.com" : "Falia Rahmazati Putri", "fatikayulianingrum@radikari.com" : "Fatika Yulianingrum", "hanifalvian@radikari.com" : "Hanif Alvian Rosyiadi", "haryoajibandera@radikari.com" : "Haryo Aji Bandera", "khusnulridho@radikari.com" : "Khusnul Ridho Moh Arif", "linamakarim@radikari.com" : "Lina Makarim", "lovianaikasari@radikari.com" : "Loviana Ika Sari", "martinus.beko@radikari.com" : "Martinus Beko", "muhammad.anwarudin@radikari.com" : "Muhammad Anwarudin", "n.agungpermana@radikari.com" : "Nuro'is Agung Permana", "nadiairana@radikari.com" : "Nadia Irana", "nonyaffi@radikari.com" : "Nony mariani affi", "paraldisbastian@radikari.com" : "Paraldi Sbastian Bagaskara", "radenardimas@radikari.com" : "Raden Ardimas Suryo Prabowo", "rantisulistiyoningsih@radikari.com" : "Ranti Sulistiyoningsih", "rasiti@radikari.com" : "Raden Ajeng Siti Nurul Erika Farida", "reckybranada@radikari.com" : "Recky Branada", "saniayolanda@radikari.com" : "Sania Yolanda", "sasalanthika@radikari.com" : "Sasa Lanthika", "sitinoordiwanti@radikari.com" : "Siti Noor Diwanti", "syamsudinbadawi@radikari.com" : "Syamsudin Badawi", "tomyprabowo@radikari.com" : "Tomy Ady Prabowo", "trisiswiharjanti@radikari.com" : "Tri Siswi Harjanti", "ummihamidah@radikari.com" : "Ummi Hamidah Kartikasari", "wahyusetiawan@radikari.com" : "Wahyu Setiawan", "ahmadbukhori@radikari.com" : "Ahmad Bukhori", "albertngobut@radikari.com" : "Albert Pieter Ngobut", "danangpranowo@radikari.com" : "Danang Wahyu Pranowo", "deaningrum@radikari.com" : "Dea Ayu Septia Ningrum", "salmafebiola@radikari.com" : "Salma Silvia Febiola", "wulansyaidah@radikari.com" : "Wulan Dewi Fatikhatus Syaidah", "adeismoyowati@radikari.com" : "Ade Ismoyowati Setiyono", "agent.srg001@radikari.com" : "Hamzah Fakhrurozi", "agent.srg003@radikari.com" : "Yoditya Iqbal Respati", "agent.srg005@radikari.com" : "Dewi Murwati", "agent.srg032@radikari.com" : "Nanda Puspa Anisya", "agent.srg037@radikari.com" : "Dody Riza Permadi", "agent.srg072@radikari.com" : "Syarifudin Fathur", "agent.srg141@radikari.com" : "Fany Slamet Sulastri", "agent.srg343@radikari.com" : "Mahendra Harima Putra", "agent.srg431@radikari.com" : "Irvan Mahendra", "agent.srg438@radikari.com" : "Novan Lestari Putra", "ajengwidyaningsih@radikari.com" : "Ajeng Widyaningsih", "yunizafakmal@radikari.com" : "Yunizaf Akmal Bachtiar", "agent.srg509@radikari.com" : "Yuliana", "agent.srg512@radikari.com" : "Bakhtiar Seno Aji", "agent.srg513@radikari.com" : "Achmad Khoirul", "agent.srg514@radikari.com" : "Millenia Sofianti", "agent.srg516@radikari.com" : "Rio Hadi Susilo", "agent.srg522@radikari.com" : "Randira Aulia", "agent.srg523@radikari.com" : "Argia Asmara Dewanggoro", "agent.srg524@radikari.com" : "Adella Titah Anggita", "agent.srg527@radikari.com" : "Dias Nugroho", "agent.srg528@radikari.com" : "Dhila Muhammad Ali", "agent.srg529@radikari.com" : "Muhammad Iqbal Muhyiddin", "agent.srg530@radikari.com" : "Aulia Rachma Yanuarita", "agent.srg531@radikari.com" : "Catur Prasetya", "agent.srg532@radikari.com" : "Siti Hartinah", "agent.srg534@radikari.com" : "Anthony Wijaya", "agent.srg535@radikari.com" : "Tri Adi Wijaya", "agent.srg536@radikari.com" : "Mohammad Fikri Mudzaki", "agiar.asmara@radikari.com" : "Agiar Asmara Bagaskoro", "ahmad.fakhruri@radikari.com" : "Ahmad Ali Fakhruri", "amynaufal@radikari.com" : "Amy Naufal Falah", "anasaidatin@radikari.com " : "Ana Saidatin", "angga.ramadhan@radikari.com" : "Angga Ramadhan Sieko Putra", "arimurtiningtyas@radikari.com" : "Ari Murtiningtyas", "atiknasekha@radikari.com" : "Atik Dina Nasekha", "aysiliaratnasari@radikari.com" : "Aysilia Audy Ratnasari", "cakratriatmaja@radikari.com" : "Cakra Triatmaja", "daityo@radikari.com" : "Dhimas Adityo Nurhayat", "devipujiseptiyani@radikari.com" : "Devi Puji Septiyani", "edosaputra@radikari.com" : "Edo Hidayatullah Saputra", "fahriidhofi@radikari.com" : "Fahri Idhofi", "farahautami@radikari.com" : "Farah Alma Utami", "firdamaudy@radikari.com" : "Firda Maudy Salsabilla", "galiehsaputra@radikari.com" : "Galih Wahyu Hadi Saputra", "gilang.pratama@radikari.com" : "Gilang Dika Pratama", "gilangadi@radikari.com" : "Gilang Adi Pratama", "hafiattaqi@radikari.com" : "Hafi Attaqi Rumi", "hafidalaziz@radikari.com" : "Hafid Alaziz", "hanifalbab@radikari.com" : "Hanif Albab", "hanung@radikari.com" : "Husein Anung Anindhito", "hriefandi@radikari.com" : "Hilman Riefandi", "isnaeni.trisna@radikari.com" : "Isnaeni Trisna Rahmawati", "mahabbatulmaula@radikari.com" : "Mahabbatul Maula", "mayhusnaino@radikari.com" : "Mayhusna Ino Tantyasari", "mluqmanhakim@radikari.com" : "Muhammad Luqman Hakim Abdullah", "Muhammad.faisal@radikari.com" : "Muhammad Faisal Amri", "muhammad.mudi@radikari.com" : "M. Mudi", "muhammadvarian@radikari.com" : "Muhammad Farell Varian", "mvennesa@radikari.com" : "Mellania Vennesa Fitri", "nadya.yun@radikari.com" : "Nadya Yun Tafa Luwih", "nafidzailmi@radikari.com" : "Nafidza Ilmi Khairina", "natanielaxl@radikari.com" : "Nataniel Axl Areza", "novitaqiroati@radikari.com" : "Novita Qiroati Fadilah", "nurzilzabahri@radikari.com" : "Nur Zilza Bunayya Bahri", "nyasykur@radikari.com" : "Naufal Yasykur Odi Pindo", "oktariskasalas@radikari.com" : "Oktariska Salas Niko", "rafandi@radikari.com" : "Rosyad Afandi", "⁠reza.adjie@radikari.com" : "Reza Purnama Adjie", "riyanita.nita@radikari.com" : "Riyanita", "sarpini.narmi@radikari.com" : "Sarpini", "satryaajisepta@radikari.com" : "Satrya Aji Septa Buana", "sitichoiriyah@radikari.com" : "Siti Choiriyah", "wahyu.setyawan@radikari.com" : "Wahyu Setyawan", "yudwita.rahmana@radikari.com" : "Yudwita Rohmana", "yuliadwi@radikari.com" : "Yulia Dwi Hardyanti", "yunan.azzumardi@radikari.com" : "Yunan Azzumardi Irawan", "ahmadkarim@radikari.com" : "Ahmad Busyrol Karim", "galihsari@radikari.com" : "Galih Ayu Wulan Sari", "kalistasedha@radikari.com" : "Kalista Sia Sedha", "rizzafahlevi@radikari.com" : "Muhammad Rizza Fahlevi", "saidihusaini@radikari.com" : "Muhammad Saidi Husaini MZ", "zakyulinnuha@radikari.com" : "Muhammad Zaky Ulinnuha", "ninengrahayu@radikari.com" : "Nineng Sri Rahayu", "reggawijaya@radikari.com" : "W. Regga Wijaya", "wahyusetiyawan@radikari.com" : "Wahyu Setiyawan", "anisaprihastuti@radikari.com" : "Anisa Prihastuti", "dyahbudiarti@radikari.com" : "Dyah Retno Budiarti", "ernifransiska@radikari.com" : "Erni Fransiska", "ferdinansimanjuntak@radikari.com" : "Ferdinan Aditya Simanjuntak", "ghovuragustin@radikari.com" : "Ghovur Azrian Agustin", "iqbalodja@radikari.com" : "Iqbal Ma'Kar Dl. Odja", "mochwijaya@radikari.com" : "Moch Rizky Eka Wijaya", "mohamadbasit@radikari.com" : "Mohamad Basit", "khoirulanam@radikari.com" : "Muhammad Nur Khoirul An Am", "mutiarahmajuni@radikari.com" : "Mutia Rahmajuni", "niputunurahdika@radikari.com" : "Ni Putu Irene Pasca Nurahdika", "ridhorahmanto@radikari.com" : "Ridho Rahmanto", "verlynapratiwi@radikari.com" : "Verlyna Dian Pratiwi", "candrakusuma@radikari.com" : "Candra Jati Kusuma", "nurazizah@radikari.com" : "Nur Roid Nafiatul Azizah", "putripayapo@radikari.com" : "Putri Nirmala S Payapo"
}


In [35]:
# Print the table header
print("| ID Agent                        | Name                       |")
print("|--------------------------------|----------------------------|")

# Print each row of the table
for id_agent, name in agent_dict.items():
    print(f"| {id_agent:<30} | {name:<26} |")

| ID Agent                        | Name                       |
|--------------------------------|----------------------------|
| agent.jogja403@radikari.com    | Yunita Risang Saksana Awiati |
| agent.jogja407@radikari.com    | Wibulan Ekaning Ratri      |
| kwijaya@radikari.com           | Krisna Wijaya              |
| agent.jogja435@radikari.com    | Ryan Oktavianto            |
| agent.jogja438@radikari.com    | Chatarina Ria Septiana     |
| wiwiks@radikari.com            | Wiwik Suharti              |
| agent.jogja487@radikari.com    | Teguh Bagus Joko Wahono    |
| agent.jogja511@radikari.com    | Hesti Vera                 |
| agent.jogja517@radikari.com    | Arlinda Elsa Ulvia         |
| agent.jogja566@radikari.com    | Eri Hardani                |
| amalia.nurhayati@radikari.com  | Amalia Nurhayati           |
| agent.jogja615@radikari.com    | Puji Kasih Santika         |
| agent.jogja674@radikari.com    | Wiwid prastianto           |
| agent.jogja738@radikari.com    | Yu

In [36]:
# for idx, row in df.iterrows():
#     if row['ID AGENT'] == '':
#       if pd.isna(row['Keterangan']) or row['Keterangan'] == '':
#         df.at[idx, 'Keterangan'] = 'ID Agent Salah'
#       else:
#         df.at[idx, 'Keterangan'] = f"{row['Keterangan']} & ID Agent Salah"

#     elif row['ID AGENT'] not in agent_dict:
#         if row['NAMA AGENT'] in agent_dict.values():
#             # Replace id agent with dict id agent
#             for k, v in agent_dict.items():
#                 if v == row['NAMA AGENT']:
#                     df.at[idx, 'ID AGENT'] = k
#                     if pd.isna(row['Keterangan']) or row['Keterangan'] == '':
#                       df.at[idx, 'Keterangan'] = 'ID Agent Salah'
#                     else:
#                       df.at[idx, 'Keterangan'] = f"{row['Keterangan']} & ID Agent Salah"
#                       break
#         else:
#             if df.at[idx, 'Keterangan'] == "":
#                 df.at[idx, 'Keterangan'] = 'ID Agent & Nama Agent Salah'
#             else:
#              df.at[idx, 'Keterangan'] = f"{row['Keterangan']} & ID Agent Salah & Nama Agent Salah"

#     elif row['NAMA AGENT'] != agent_dict[row['ID AGENT']]:
#         # Replace agent name with dict agent name
#         df.at[idx, 'NAMA AGENT'] = agent_dict[row['ID AGENT']]
#         if df.at[idx, 'Keterangan'] == "":
#             df.at[idx, 'Keterangan'] = 'Nama Agent Salah'
#         else:
#          df.at[idx, 'Keterangan'] = f"{row['Keterangan']} & Nama Agent Salah"


In [37]:
for idx, row in df.iterrows():
    current_ket = df.at[idx, 'Keterangan'] if not pd.isna(df.at[idx, 'Keterangan']) else ""

    if row['ID AGENT'] == '':
        df.at[idx, 'Keterangan'] = 'ID Agent Salah' if current_ket == "" else f"{current_ket} & ID Agent Salah"

    elif row['ID AGENT'] not in agent_dict:
        if row['NAMA AGENT'] in agent_dict.values():
            match_id = next((k for k, v in agent_dict.items() if v == row['NAMA AGENT']), None)
            if match_id:
                df.at[idx, 'ID AGENT'] = match_id
                ori_df.at[idx, 'ID AGENT'] = match_id
                df.at[idx, 'Keterangan'] = 'ID Agent Salah' if current_ket == "" else f"{current_ket} & ID Agent Salah"
        else:
            df.at[idx, 'Keterangan'] = 'ID Agent & Nama Agent Salah' if current_ket == "" else f"{current_ket} & ID Agent Salah & Nama Agent Salah"

    elif row['NAMA AGENT'] != agent_dict.get(row['ID AGENT'], ''):
        df.at[idx, 'NAMA AGENT'] = agent_dict[row['ID AGENT']]
        ori_df.at[idx, 'NAMA AGENT'] = agent_dict[row['ID AGENT']]
        df.at[idx, 'Keterangan'] = 'Nama Agent Salah' if current_ket == "" else f"{current_ket} & Nama Agent Salah"

In [ ]:
filtered_data = df[df['Keterangan'].notnull()]
filtered_data.head(10)

## Check Modified Date & Working Date

In [39]:
# Create a custom function to check conditions and update "Keterangan" column
def update_keterangan(row):
    if pd.isna(row['Modified Date']):
      if pd.isna(row['Keterangan']):
        return 'modified date salah'
      else:
        return f"{row['Keterangan']} & modified date salah"

    elif row['Modified Date'].date() != row['WorkingDate'].date():
        if pd.isna(row['Keterangan']):
            return 'working date salah'
        else:
            return f"{row['Keterangan']} & working date salah"
    else:
        return row['Keterangan']

# Apply the custom function to update "Keterangan" column
df['Keterangan'] = df.apply(update_keterangan, axis=1)

#Change The Data

##Mengganti Nomor Telpon

In [40]:
pd.set_option('display.max_rows', None)
telpon = df[df['Keterangan'].str.contains('telpon', case=False, na=False)]
telpon[['ORDER ID','NO. HP', 'Modified Date','WorkingDate','Keterangan']]


,ORDER ID,NO. HP,Modified Date,WorkingDate,Keterangan
392792,20240823111215800000DANAW3ID546503238657,45528,2024-08-23 22:06:29,2024-08-23,no telpon salah
985411,20240819111215800000DANAW3ID128702090005,NaN,2024-08-19 12:25:53,2024-08-19,no telpon salah


In [41]:
# Define a base URL
base_url = "https://custmng.saas.dana.id/user/kyc/detail.htm?menuKey=4&kycOrderId="

# Function to concatenate URL with order ID and make it a full link
def create_clickable_link(order_id):
    full_url = (f"{base_url}{order_id}&failedReason=There%20was%20a%20problem%20while%20upgrading."
                "&submissionDate=1669465310788&modifiedDate=1669465320278"
                "&orderStatus=FAILED&loginId=62-85156229291&userNickname=MUHAMMAD%20ARIES%20RAMADHAN")
    return f'<a href="{full_url}" target="_blank">Link</a>'

# Adding a new column 'Link' that contains clickable URLs
telpon['Link'] = telpon['ORDER ID'].apply(create_clickable_link)

# Select only the 'ORDER ID' and 'Link' columns for display
display_columns = telpon[['ORDER ID', 'Link']]

# Display the DataFrame with clickable links (now with the index)
display(display_columns.style.format({'Link': lambda x: x}).set_properties(**{'text-align': 'left'}))

#========================== to print
# Convert the 'ORDER ID' column to a list
order_id_list = telpon['ORDER ID'].tolist()

# Join the list items into a single string separated by commas
order_ids_string = ", ".join(order_id_list)

print("")
# Print the result
print(order_ids_string)

,ORDER ID,Link
392792,20240823111215800000DANAW3ID546503238657,Link
985411,20240819111215800000DANAW3ID128702090005,Link



20240823111215800000DANAW3ID546503238657, 20240819111215800000DANAW3ID128702090005


In [42]:

def change_telpon_bulk(df):
    changed_rows = set()

    # Prompt the user for Order IDs
    order_ids_input = input("Enter all Order IDs separated by commas (or type 'No' to stop): ").strip()

    # Check if the user wants to stop
    if order_ids_input.lower() == 'no':
        print("Stopping the process.")
        return

    # Split the input into a list of Order IDs
    order_ids = [order_id.strip() for order_id in order_ids_input.split(',')]

    for order_id_to_change in order_ids:
        # Prompt the user for the new Status value for each Order ID
        new_status_value = input(f"Enter the new No Telepon value for ORDER ID {order_id_to_change}: ").strip()

        # Locate the row with the specified "ORDER ID"
        changed_row = df[df['ORDER ID'] == order_id_to_change].index

        if changed_row.empty:
            print(f"\nNo row found for ORDER ID: {order_id_to_change}")
        else:
            # Update the "STATUS" value
            df.loc[changed_row, 'NO. HP'] = new_status_value
            ori_df.loc[changed_row, 'NO. HP'] = new_status_value
            # Add the changed row to the set
            changed_rows.add(changed_row[0])

            # Display only the 'ORDER ID' and 'STATUS' columns for the changed row
            # print(f"\nChanged Row for ORDER ID {order_id_to_change}:\n{df.loc[changed_row, ['ORDER ID', 'STATUS']]}")

    # Display all changed rows after stopping the process
    if changed_rows:
        print("\nAll Changed Rows:")
        #print(df.loc[changed_rows, ['ORDER ID', 'NO. HP']])

# Example usage:
change_telpon_bulk(df)

Enter all Order IDs separated by commas (or type 'No' to stop): 20240823111215800000DANAW3ID546503238657, 20240819111215800000DANAW3ID128702090005
Enter the new No Telepon value for ORDER ID 20240823111215800000DANAW3ID546503238657: 6281573415963
Enter the new No Telepon value for ORDER ID 20240819111215800000DANAW3ID128702090005: 6283870755340

All Changed Rows:


## Mengganti Status

In [43]:
pd.set_option('display.max_rows', None)
status = df[(df['Keterangan'].str.contains('check status', na=False)) &
            (df['Modified Date'].notnull()) &
            (~df['STATUS'].str.contains('REJECT BY SYSTEM', na=False))
            ]
sorted_status = status.sort_values(by='STATUS')
# sorted_status[['ID AGENT', 'ORDER ID', 'NO. HP', 'STATUS', 'Reason', 'Modified Date', 'WorkingDate', 'Keterangan']]


In [ ]:
# Define a base URL
base_url = "https://custmng.saas.dana.id/user/kyc/detail.htm?menuKey=4&kycOrderId="

# Function to concatenate URL with order ID and make it a full link
def create_clickable_link(order_id):
    full_url = (f"{base_url}{order_id}&failedReason=There%20was%20a%20problem%20while%20upgrading."
                "&submissionDate=1669465310788&modifiedDate=1669465320278"
                "&orderStatus=FAILED&loginId=62-85156229291&userNickname=MUHAMMAD%20ARIES%20RAMADHAN")
    return f'<a href="{full_url}" target="_blank">Link</a>'

# Adding a new column 'Link' that contains clickable URLs
sorted_status['Link'] = sorted_status['ORDER ID'].apply(create_clickable_link)

# Select only the 'ORDER ID' and 'Link' columns for display
display_columns = sorted_status[['NAMA AGENT','NO. HP','ORDER ID', 'STATUS', 'Reason','Status Revoke','Link','Modified Date', 'WorkingDate', 'Keterangan']]

# Display the DataFrame with clickable links (now with the index)
display(display_columns.style.format({'Link': lambda x: x}).set_properties(**{'text-align': 'left'}))

print("Jumlah Row:  ",len(sorted_status))

In [45]:
# Dictionary of changes (ORDER ID as keys, values as dictionaries of new STATUS and KETERANGAN)
changes = {

    '20240822111215800000DANAW3ID474704393250': {'STATUS': 'REJECT', 'Keterangan': 'finding salah status'},

    '20240822111215800000DANAW3ID140303986144': {'STATUS': 'CHECK', 'Keterangan': 'status revoke salah'},

    '20240824111215800000DANAW3ID084305076093': {'STATUS': 'CHECK', 'Keterangan': 'status revoke salah'},

    '20240824111215800000DANAW3ID093705112105': {'STATUS': 'CHECK', 'Keterangan': 'reason salah'},

    '20240823111215800000DANAW3ID161904944288': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    '20240823111215800000DANAW3ID646004194771': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'}

    # '20240823111215800000DANAW3ID555303486240': {'STATUS': 'CHECK', 'Keterangan': 'status revoke salah'},

    # '20240817111215800000DANAW3ID362699177524': {'STATUS': 'CHECK', 'Keterangan': 'status revoke salah'}

    # '20240731111215800000DANAW3ID546886982239': {'STATUS': 'CHECK', 'Keterangan': 'finding salah status'}

    # '20240725111215800000DANAW3ID237083753702': {'STATUS': 'REVOKED', 'Keterangan': 'reason salah'},

    # '': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'}


}
# Apply the changes in one line
for order_id, values in changes.items():
    df.loc[df['ORDER ID'] == order_id, ['STATUS', 'Keterangan']] = [values['STATUS'], values['Keterangan']]
    ori_df.loc[ori_df['ORDER ID'] == order_id, ['STATUS', 'Keterangan']] = [values['STATUS'], values['Keterangan']]


Function For Checking

In [ ]:
# pd.set_option('display.max_rows', None)
# status = df[df['ORDER ID'].str.contains('20231201111215800000DANAW3ID970814217454',na=False)]
# status[['ORDER ID','STATUS','Keterangan']]



#Processing Data 2

In [46]:
# Replace NaN with empty string and convert to string
df['Reason'] = df['Reason'].fillna('').astype(str)
df['New Remarks'] = df['New Remarks'].fillna('').astype(str)
df['KTP Defect >1'] = df['KTP Defect >1'].fillna('').astype(str)
df['Status Revoke'] = df['Status Revoke'].fillna('').astype(str)
df['Reason Revoke'] = df['Reason Revoke'].fillna('').astype(str)
df['New Remarks Revoke'] = df['New Remarks Revoke'].fillna('').astype(str)

In [47]:
# Remove Whitespace
df['Status Revoke'] = df['Status Revoke'].str.strip()
df['Reason Revoke'] = df['Reason Revoke'].str.strip()
df['New Remarks Revoke'] = df['New Remarks Revoke'].str.strip()

In [ ]:
df.tail(2)

##Check Doc Type


In [49]:
#Doc Type list
doc_type_list = [
  'KTP',
  'DRIVING_LICENSE'
]

In [50]:
def doc_type (row):
    if row['Doc Type'].lower() not in map(str.lower, doc_type_list) or pd.isna(row['Doc Type']):
      if pd.isna(row['Keterangan']):
        return 'doc type salah'
      else:
        return f"{row['Keterangan']} & doc type salah"

    else:
        return row['Keterangan']

In [51]:
df['Keterangan'] = df.apply(doc_type, axis=1)

##Check Status Revoke, Reject Reason, New Remarks Reason

In [52]:
# Select rows where STATUS is 'REJECT' or 'REVOKE'
reject_revoke = df[(df['STATUS'] == 'REJECT') | (df['STATUS'] == 'REVOKED')]

In [53]:
#Ada problem dengan reason revoke inconsisten, mengisi ulang nilai baru dengan kategori ini
pd.set_option('display.max_rows', None)
reject_revoke.loc[reject_revoke['Reason Revoke'].str.contains('INCONSISTENT', case=False, na=False), 'Reason Revoke'] = 'INCONSISTENT KTP (KTP TIDAK KONSISTEN)'
# status[['ORDER ID','STATUS','Status Revoke','Reason Revoke','New Remarks Revoke','Keterangan']]

In [54]:
#Ada problem dengan reason revoke inconsisten, mengisi ulang nilai baru dengan kategori ini
pd.set_option('display.max_rows', None)
df.loc[df['Reason Revoke'].str.contains('INCONSISTENT', case=False, na=False), 'Reason Revoke'] = 'INCONSISTENT KTP (KTP TIDAK KONSISTEN)'
# status[['ORDER ID','STATUS','Status Revoke','Reason Revoke','New Remarks Revoke','Keterangan']]

In [55]:
#reject reason list
valid_reasons_revoke = [
    'YOUR ID CARD IS NOT EKTP/KTP',
    'YOUR ID CARD PICTURE IS NOT CLEAR',
    'THE ID NUMBER THAT YOU TYPED DOES NOT MATCH WITH THE ID NUMBER IN THE PICTURE',
    'YOUR ID CARD IS FAKE OR YOUR PHOTO IN THE ID IS FAKE',
    "YOUR SELFIE DOESN'T MATCH WITH ID PHOTO",  # Use double quotes around this string
    'YOUR ID CARD IS ALREADY EXPIRED',
    'YOUR KTP BACKGROUND COLOR DOES NOT MATCH',
    'YOU REQUEST TO REVOKE KYC',
    'KTP ON SCREEN (KTP ADA DI LAYAR)',
    'KTP DAMAGE (RUSAK)',
    'KTP PRINT OUT (KTP CETAKAN)',
    'INCONSISTENT KTP (KTP TIDAK KONSISTEN)',
    'TERDAPAT LEBIH DARI SATU ORANG PADA FOTO SELFIE',
    'FETCHING'
]

valid_reasons_reject = [
"Reject document doesn't match the specified type (Dokumen tidak sesuai dengan jenis yang ditentukan)",
"Reject document has expired (Dokumen kedaluwarsa)",
"Reject document is not clear and blurred (Dokumen tidak jelas dan buram)",
"Reject document is not readable (Dokumen tidak dapat dibaca)",
"Reject document is overexposed (Dokumen terlalu terpapar cahaya)",
"Reject document is underexposed (Dokumen terlalu gelap)",
"Reject document number doesn't match the user's input DATA CLEAR (Nomor dokumen tidak sesuai dengan yang diketik user - Data Jelas)",
"Reject document number doesn't match the user's input DATA UNCLEAR (Nomor dokumen tidak sesuai dengan yang diketik user - Data tidak jelas)",
"Reject picture in the document is edited (Gambar dalam dokumen diedit)",
"Reject selfie doesn't match the photo in the document (Selfie tidak sesuai dengan foto dalam dokumen)",
"Reject Supporting document type is not acceptable (SIM, Passport, KK, Ijazah, Buku Nikah). (Jenis dokumen pendukung tidak sesuai ketentuan (SIM, Paspor, KK, Buku Nikah, Ijazah)"
]


In [56]:
#new remarks revoke
remark_list = [
    'Abnormal Background (Background foto KTP tidak seperti biasa)',
    'Background KTP doesn’t match (Background Foto KTP Tidak Sesuai)',
    'Biometric Score <6',
    'Biometric Score <8',
    'DOB Not Match With DOP Id Card (TTL Tidak Sesuai Dengan Tahun Pembuatan)',
    'Document is Overexposure (Foto Dokumen Terlalu Terang)',
    'Edited Id Card (KTP Editan)',
    'Id Card Damage (KTP Rusak)',
    'Id Card Image Blur (KTP Blur)',
    'Inconsistent Data, > 4 Id Card (Data Inkonsisten, History lebih dari 4)',
    'On Screen Id Card (KTP On Screen)',
    'Photo On Id Card Taped (Foto KTP Tempelan)',
    'Photocopy ID Card (KTP Fotokopi)',
    'Print Id Card (KTP Print)',
    'Selfie Doesn’t Match With Id Card (Selfie tidak sesuai)',
    #'Support Document Inappropriate (Dokumen pendukung tidak sesuai)', #delet ini 2024
    'Support Document Vulgar (Dokumen Pendukung Vulgar)',
    'Support Document Invalid (Dokumen Pendukung Tidak Sesuai)',
    'Suspicious ID Network High Risk',
    'Suspicious Ilegal Letters High Risk',
    'Suspicious Screen Retaken High Risk',
    'Suspicious Velocity High Risk',
    'The Id Number That You Typed Does Not Match With The Id Number In The Picture (NIK Typo)',
    'Your Id Card Is Already Expired (KTP Expired)',
    'Your Id Card Is Not Ektp/Ktp (Bukan E-KTP/KTP)',
    'Your Id Card Picture Is Not Clear (Tertutup, Terkena Noda & Ditebalkan)'
]

In [57]:
def status_revoke(row):
    if row['STATUS'] in ['REJECT', 'REVOKED'] and pd.isna(row['Status Revoke']) or row['Status Revoke'] not in ['REVOKED', 'REVOKED & Apply KYC']:
      # Update 'Status Revoke' to 'REVOKED'
      df.loc[row.name, 'Status Revoke'] = 'REVOKED'
      if pd.isna(row['Keterangan']):
        return 'status revoke salah'
      else:
        return f"{row['Keterangan']} & status revoke salah"
    else:
      return row['Keterangan']

In [58]:
def reason_revoke (row):
    if row['STATUS'].lower() in ['reject', 'revoked'] and pd.isna(row['Reason Revoke']):
    #if row['STATUS'] in ['REJECT', 'REVOKED'] and pd.isna(row['Reason Revoke']) or row['Reason Revoke'] not in valid_reasons:
      if pd.isna(row['Keterangan']):
        return 'reason revoke salah'
      else:
        return f"{row['Keterangan']} & reason revoke salah"

    elif row['STATUS'].lower() in ['reject'] and row['Reason Revoke'].lower() not in map(str.lower, valid_reasons_reject):
      if pd.isna(row['Keterangan']):
        return 'reason revoke salah'
      else:
        return f"{row['Keterangan']} & reason revoke salah"

    elif row['STATUS'].lower() in ['revoke'] and row['Reason Revoke'].lower() not in map(str.lower, valid_reasons_revoke):
      if pd.isna(row['Keterangan']):
        return 'reason revoke salah'
      else:
        return f"{row['Keterangan']} & reason revoke salah"

    else:
        return row['Keterangan']

In [59]:
def nr_revoke (row):
    if row['STATUS'].lower() in ['reject', 'revoked'] and pd.isna(row['New Remarks Revoke']) or row['New Remarks Revoke'].lower() not in map(str.lower, remark_list):
    #if row['STATUS'] in ['REJECT', 'REVOKED'] and pd.isna(row['New Remarks Revoke']) or row['New Remarks Revoke'] not in remark_list:
      if pd.isna(row['Keterangan']):
        return 'new remarks revoke salah'
      else:
        return f"{row['Keterangan']} & new remarks revoke salah"

    else:
        return row['Keterangan']

In [60]:
# Apply the custom function to update "Keterangan" column
reject_revoke['Keterangan'] = reject_revoke.apply(status_revoke, axis=1)
reject_revoke['Keterangan'] = reject_revoke.apply(reason_revoke, axis=1)
reject_revoke['Keterangan'] = reject_revoke.apply(nr_revoke, axis=1)

In [ ]:
x = reject_revoke[reject_revoke['Keterangan'].str.contains('Revoke', case=False, na=False)]
x[['ORDER ID','NO. HP','STATUS','Status Revoke','Reason Revoke','New Remarks Revoke','Keterangan']]

#Finishing Data

In [62]:
# Remove values in 'Keterangan' column where 'STATUS' is "REJECT BY SYSTEM"
df.loc[df['STATUS'] == 'REJECT BY SYSTEM', 'Keterangan'] = np.nan
df.loc[df['Reason'] == 'REJECT BY SYSTEM', 'Keterangan'] = np.nan

In [63]:
# Combine the values from both dataframes into a single column
df['Keterangan'] = df['Keterangan'].combine_first(reject_revoke['Keterangan'])
ori_df['Keterangan'] = ori_df['Keterangan'].combine_first(df['Keterangan'])
ori_df['STATUS'] = ori_df['STATUS'].combine_first(df['STATUS'])
ori_df['ID AGENT'] = ori_df['ID AGENT'].combine_first(df['ID AGENT'])
ori_df['NAMA AGENT'] = ori_df['NAMA AGENT'].combine_first(df['NAMA AGENT'])


In [64]:
# # Perform left join to bring updated "NO. HP" values
# temp_df = ori_df.merge(df[['ORDER ID', 'NO. HP']], on='ORDER ID', how='left', suffixes=('', '_updated'))

# # Update "NO. HP" in ori_df with the values from df
# ori_df['NO. HP'] = temp_df['NO. HP_updated'].combine_first(temp_df['NO. HP'])

In [ ]:
ori_df.tail()

In [66]:
# Get a list of columns containing 'Unnamed'
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]

# Drop the columns with 'Unnamed' in their names
df = df.drop(columns=unnamed_columns)
ori_df = ori_df.drop(columns=unnamed_columns)


## Backup Revoke

In [ ]:
# Get user input for WorkingDate
user_input = input("Enter WorkingDate (yyyy-mm-dd) : ")

# Filter DataFrame based on conditions
Revoke = df[(df['STATUS'].isin(['REVOKED', 'REJECT'])) & (df['WorkingDate'] == user_input)]

# Sort the filtered DataFrame by 'Reason Revoke' ascending
sorted_df = Revoke.sort_values(by='Reason', ascending=True)

# Format the 'Modified Date' column to 'dd-mm-yyyy hh:mm:ss'
sorted_df['Modified Date'] = sorted_df['Modified Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
sorted_df['WorkingDate'] = sorted_df['WorkingDate'].dt.strftime('%d-%m-%Y')

# First, convert the 'Face Score' column to numeric (float)
sorted_df['Face Score'] = pd.to_numeric(sorted_df['Face Score'], errors='coerce')

# Now, convert to integer
sorted_df['Face Score'] = sorted_df['Face Score'].astype('Int64')
#sorted_df['Face Score'] = sorted_df['Face Score'].astype('str')

# Replace NaN or NaT with blank (empty string)
#sorted_df.replace({np.nan: '', pd.NaT: ''}, inplace=True)

# Sort the filtered DataFrame by 'Reason Revoke' ascending
sorted_df = Revoke.sort_values(by='Reason', ascending=True)

sorted_df.head()

In [68]:
sorted_df['D'] = np.nan
new_order = ['ORDER ID', 'NO. HP', 'Doc Type','STATUS','D','D','ID AGENT','NAMA AGENT','D','Support DOC','Face Score','OCR','Biometric Score','Modified Date','WorkingDate','KTP Defect >1','NAMA AGENT','WorkingDate','Status Revoke','Reason Revoke','Reason','New Remarks Revoke']
sorted_dfx = sorted_df[new_order]

In [69]:
# from google.colab import files
# # Save DataFrame to an Excel file
sorted_dfx.to_excel('Revoke 24 August.xlsx', index=False)
# # Download the file
files.download('Revoke 24 August.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Monitoring Pengerjaan Agent

In [70]:
# Filter rows based on conditions and set 'Keterangan' to null
df.loc[(df['STATUS'].isnull()) & (df['Modified Date'].isnull()) & (df['WorkingDate'] == end_date), 'Keterangan'] = None

In [71]:
# Set display option to show all rows
pd.set_option('display.max_rows', None)

In [ ]:
filtered_df = df[
    # (df['STATUS'].notnull()) &
    # (df['Reason'].notnull()) &
    # (df['Modified Date'].notnull()) &
    (df['Keterangan'].notnull() & ~df['Keterangan'].str.contains('finding', case=False, na=False))
]

# Format the 'Modified Date' column to 'dd-mm-yyyy hh:mm:ss'
filtered_df['Modified Date'] = filtered_df['Modified Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
filtered_df['WorkingDate'] = filtered_df['WorkingDate'].dt.strftime('%d-%m-%Y')

# First, convert the 'Face Score' column to numeric (float)
filtered_df['Face Score'] = pd.to_numeric(filtered_df['Face Score'], errors='coerce')

# Now, convert to integer
filtered_df['Face Score'] = filtered_df['Face Score'].astype('Int64')
filtered_df['Face Score'] = filtered_df['Face Score'].astype('str')

# Replace NaN or NaT with blank (empty string)
filtered_df.replace({np.nan: '', pd.NaT: ''}, inplace=True)

# Drop duplicates in the 'ORDER ID' column
filtered_df_no_duplicates = filtered_df.drop_duplicates(subset='ORDER ID')

# Remove values in 'Keterangan' column where 'STATUS' is "REJECT BY SYSTEM"
filtered_df_no_duplicates.loc[filtered_df_no_duplicates['STATUS'] == 'REJECT BY SYSTEM', 'Keterangan'] = ''
filtered_df_no_duplicates.loc[filtered_df_no_duplicates['Reason'] == 'REJECT BY SYSTEM', 'Keterangan'] = ''

filtered_df_no_duplicates.head(10)



In [73]:
filtered_df_no_duplicates.to_excel('monitoring.xlsx', index=False)
# # # Download the file
files.download('monitoring.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Finding

In [74]:
# Apply filters
finding = df[df['Keterangan'].str.contains('finding', case=False, na=False)]

# First, convert the 'Face Score' column to numeric (float)
finding['Face Score'] = pd.to_numeric(finding['Face Score'], errors='coerce')

# Now, convert to integer
finding['Face Score'] = finding['Face Score'].astype('Int64')
finding['Face Score'] = finding['Face Score'].astype('str')

finding.replace({np.nan: '', pd.NaT: ''}, inplace=True)
finding

,ORDER ID,NO. HP,Doc Type,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Remarks Zolos Incomplete,Keterangan
251717,20240822111215800000DANAW3ID474704393250,6283139575121,KTP,REJECT,BACKGROUND TIDAK SESUAI TTL,,khusnulridho@radikari.com,Khusnul Ridho Moh Arif,2024-08-24 18:54:16,KK,71,Sesuai,0,2024-08-24,,REVOKED,,,,finding salah status


In [75]:
finding.to_excel('finding.xlsx', index=False)
# # Download the file
files.download('finding.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Data Process

In [76]:
# Filter out rows where 'ID AGENT' or 'NAMA AGENT' is null
filtered_df = ori_df.dropna(subset=['ID AGENT', 'NAMA AGENT','STATUS'])

# Count occurrences of each ORDER ID grouped by Working Date
order_id_counts = filtered_df.groupby(['WorkingDate'])['ORDER ID'].nunique().reset_index(name='Count')
order_id_counts

,WorkingDate,Count
0,2024-08-01,44910
1,2024-08-02,50112
2,2024-08-03,46188
3,2024-08-04,47655
4,2024-08-05,47143
5,2024-08-06,44536
6,2024-08-07,49242
7,2024-08-08,51493
8,2024-08-09,47185
9,2024-08-10,45431


##All Data

#Save Data

In [ ]:
# Format the 'Modified Date' column to 'dd-mm-yyyy hh:mm:ss'
# ori_df['Modified Date'] = ori_df['Modified Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
# ori_df['WorkingDate'] = ori_df['WorkingDate'].dt.strftime('%d-%m-%Y')

# Format the 'Modified Date' column to 'dd-mm-yyyy hh:mm:ss'
ori_df['Modified Date'] = ori_df['Modified Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
ori_df['WorkingDate'] = ori_df['WorkingDate'].dt.strftime('%d-%m-%Y')

# First, convert the 'Face Score' column to numeric (float)
ori_df['Face Score'] = pd.to_numeric(ori_df['Face Score'], errors='coerce')
# Now, convert to integer
ori_df['Face Score'] = ori_df['Face Score'].astype('Int64')
ori_df['Face Score'] = ori_df['Face Score'].astype('str')

# Replace NaN or NaT with blank (empty string)
ori_df.replace({np.nan: '', pd.NaT: ''}, inplace=True)

# Remove duplicates based on 'ORDER ID' and keep the first occurrence
ori_df = ori_df.drop_duplicates(subset='ORDER ID', keep='first')

ori_df.head(5)

Save 1 File

In [ ]:
# from google.colab import files
# # Save DataFrame to an Excel file
ori_df.to_excel('Check1.xlsx',na_rep='', index=False)

# # Download the file
files.download('Check1.xlsx')

Save 2 File

In [78]:
excel_path = '/content/'

# Specify the chunk size (number of rows per Excel file)
chunk_size = len(ori_df) // 2  # Split into two equal parts

# Function to save chunks of DataFrame to Excel
def save_to_excel(dataframe, suffix):
    excel_file = f'{excel_path}Check{suffix}.xlsx'
    dataframe.to_excel(excel_file, index=False)
    print(f"Excel file {excel_file} saved successfully.")

# Split ori_df into two chunks and save to separate Excel files
Check_a = ori_df.iloc[:chunk_size]
Check_b = ori_df.iloc[chunk_size:]

save_to_excel(Check_a, '1')
save_to_excel(Check_b, '2')




Excel file /content/Check1.xlsx saved successfully.
Excel file /content/Check2.xlsx saved successfully.


In [79]:
# # Download the file
files.download('Check1.xlsx')
files.download('Check2.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Save 3 File

In [ ]:
excel_path = '/content/'

# Specify the chunk size (number of rows per Excel file)
chunk_size = len(ori_df) // 3  # Split into three equal parts

# Function to save chunks of DataFrame to Excel
def save_to_excel(dataframe, suffix):
    excel_file = f'{excel_path}Check{suffix}.xlsx'
    dataframe.to_excel(excel_file, index=False)
    print(f"Excel file {excel_file} saved successfully.")

# Split ori_df into three chunks and save to separate Excel files
Check_a = ori_df.iloc[:chunk_size]
Check_b = ori_df.iloc[chunk_size:2*chunk_size]
Check_c = ori_df.iloc[2*chunk_size:]

save_to_excel(Check_a, '1')
save_to_excel(Check_b, '2')
save_to_excel(Check_c, '3')


In [ ]:
# # Download the file
files.download('Check1.xlsx')
files.download('Check2.xlsx')
files.download('Check3.xlsx')